## Utilizando RandomForest para predições de churn 

<i>Churn é uma métrica utilizada para verificar clientes que deixaram de utilizar determinado serviço</i>

In [1]:
!pip install -q -U watermark


# pip        >  gerenciador de pacotes python
# install    >  comando para instalar um novo pacote
# -q         >  para que os eventos durante instalação não sejam exibidos (-q = quiet)
# -U         >  para que toda a vez em que o processo de instalação ocorrer o pip busque a ultima versão do pacote
# watermark  >  pacote python para exibir versão, data da execução e informações do hardware

#### Importação de pacotes

In [2]:
import joblib
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

#### Configuração para tamanho de gráficos

In [ ]:
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = [6, 5]

#### Determinando dados a serem utilizados pelo pacote watermark

In [3]:
%reload_ext watermark
%watermark -a "Octávio Duarte"

Author: Octávio Duarte



#### Carregando os dados

In [5]:
base = pd.read_csv('dataset.csv')

In [7]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Idade              1000 non-null   int64  
 1   UsoMensal          1000 non-null   int64  
 2   Plano              1000 non-null   object 
 3   SatisfacaoCliente  1000 non-null   int64  
 4   TempoContrato      1000 non-null   object 
 5   ValorMensal        1000 non-null   float64
 6   Churn              1000 non-null   int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 54.8+ KB
